In [14]:
import pandas as pd
import numpy as np

In [15]:
%cd /Users/alex/PETs-Experiment

/Users/alex/PETs-Experiment


In [16]:
import PETs_Experiment

In [17]:
loader = PETs_Experiment.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [18]:
df = loader.data

In [19]:
metadata = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'numerical', 'na_percentage': 0.0},
                'pir': {'type': 'numerical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [20]:
md1 = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [21]:
pm = PETs_Experiment.Processor_Manager(metadata)

In [22]:
test = pm.get_config(['gen', 'age'])

In [23]:
from PETs_Experiment.Preprocessor.Missingist_Drop import Missingist_Drop


test['gen']['missingist'] = Missingist_Drop(df)
test['gen']['encoder'] = None

test['age']['missingist'] = Missingist_Drop(df)
test['age']['outlierist'] = None
test['age']['scaler'] = None

In [24]:
pm.set_config(test)

In [27]:
pm.update_config({'age': {'missingist': Missingist_Drop(df)}, 'gen':{'missingist': Missingist_Drop(df)}})

In [30]:
type(pm._config['gen']['missingist'])

PETs_Experiment.Preprocessor.Missingist_Drop.Missingist_Drop